In [ ]:
import numpy as np
import pandas as pd

pd.options.display.max_columns = None
pd.options.display.max_rows = None

import requests
from bs4 import BeautifulSoup
import time

In [ ]:
url = "https://www.basketball-reference.com/leagues/NBA_2020_games-april.html"
page = pd.read_html(url)

In [ ]:
# season = 2022
# month = "october"
# base_url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"

In [ ]:
# page = requests.get("https://www.basketball-reference.com/leagues/NBA_2014_games.html")
# print(page.status_code)
# soup = BeautifulSoup(page.content, "html.parser")

# filter_div = soup.find('div', class_='filter')
# filter_div

In [ ]:
# base_url = "https://www.basketball-reference.com"
# a = [base_url + link["href"] for link in filter_div.find_all("a")]
# a

In [ ]:
for season in range(2008, 2009):
    print(season)

In [ ]:
season = ""
base_url = "https://www.basketball-reference.com"

dfs = []
for season in range(2008, 2025):
# for season in range(2008, 2010):
    time.sleep(10)
    print(season)
    temp_url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
    page = requests.get(temp_url)
    # print(page.status_code)
    soup = BeautifulSoup(page.content, "html.parser")
    filter_div = soup.find("div", class_="filter")
    if filter_div is None:
        continue

    links = [base_url + link["href"] for link in filter_div.find_all("a")]
    is_playoffs = False
    shape_string = ""
    playoff_ind = 1_000_000
    for l in links:
        time.sleep(3)
        # print(l)
        try:
            temp_df = pd.read_html(l)[0]
            temp_df['Notes'] = temp_df['Notes'].astype(str)

            if temp_df[temp_df["Date"].str.lower() == "playoffs"].shape[0] > 0:
                is_playoffs = True
                playoff_ind = temp_df[temp_df["Date"].str.lower() == "playoffs"].index[0]
                temp_df["is_playoff"] = np.where(temp_df.index >= playoff_ind, 1, 0)
            elif temp_df[temp_df["Notes"].str.lower() == "play-in game"].shape[0] > 0:
                is_playoffs = True
                playoff_ind = temp_df[temp_df["Notes"].str.lower() == "play-in game"].index[0]
                temp_df["is_playoff"] = np.where(temp_df.index >= playoff_ind, 1, 0)
            elif is_playoffs:
                temp_df["is_playoff"] = 1
            else:
                temp_df["is_playoff"] = 0
            # if temp_df[temp_df["Date"].str.lower() == "playoffs"].shape[0] > 0:
            #     is_playoffs = True
            #     playoff_ind = temp_df[temp_df["Date"].str.lower() == "playoffs"].index[
            #         0
            #     ]
            #     temp_df["is_playoff"] = np.where(temp_df.index >= playoff_ind, 1, 0)
            # elif temp_df[temp_df["Notes"].str.lower() == "play-in game"]
            # elif is_playoffs:
            #     temp_df["is_playoff"] = 1
            # else:
            #     temp_df["is_playoff"] = 0

            temp_df["season_end_year"] = season
            temp_df["season_url"] = temp_url
            temp_df["link"] = l
            shape_string += " " + str(temp_df.shape)
        except Exception as e:
            print(str(e))
            print(f"error: {season} {l}")

        dfs.append(temp_df)

    print(f"season: {season}, status_code: {page.status_code}, shapes: {shape_string}")

master = pd.concat(dfs).reset_index(drop=True)
print(master.shape)
master.head()

In [ ]:
# final cleaning steps
master["season_start_year"] = master["season_end_year"] - 1
master["date"] = pd.to_datetime(master["Date"].str.split(",", n=1).str[1])

In [ ]:
print(master.shape)
master.head()

In [ ]:
assert False

In [ ]:
# if seasons are range(2008, 2025) then it should be 21_690 rows
print(master.shape)
master.head()

In [ ]:
# index is unique
print(master.shape)
print(len(set(master.index)))

In [ ]:
# number of playoff games
master.pivot_table(
    index=['season_start_year'],
    values=["is_playoff"],
    aggfunc=sum
)

In [ ]:
# mark the playoff games
playoffs_index = master.reset_index()[master.reset_index()["Date"] == "Playoffs"].index  # [0]
playoffs_index

In [ ]:
# a = master.copy()
# a[a['Notes']== "Play-In Game"].head()


In [ ]:
# a_ind = a[a['Notes']== "Play-In Game"].index
# print(a_ind)


# a['is_playoffs'] = 1

# a.head()


In [ ]:
# note that this includes playoff games
master[master["Start (ET)"] != "Playoffs"].to_csv(
    "../data/game_times_bask_ref_2008-2024.csv", index=False
)
print(time.ctime())

In [ ]:
len(set(master.index))

In [ ]:
master.tail(15)

In [ ]:
import numpy as np

blah = pd.DataFrame(
    {
        "Date": {
            0: "Tue, Oct 30, 2007",
            1: "Tue, Oct 30, 2007",
            2: "Tue, Oct 30, 2007",
            3: "Wed, Oct 31, 2007",
            4: "Wed, Oct 31, 2007",
        },
        "Start (ET)": {0: "8:00p", 1: "10:30p", 2: "10:30p", 3: "7:00p", 4: "7:00p"},
        "Visitor/Neutral": {
            0: "Portland Trail Blazers",
            1: "Utah Jazz",
            2: "Houston Rockets",
            3: "Washington Wizards",
            4: "Milwaukee Bucks",
        },
        "PTS": {0: 97, 1: 117, 2: 95, 3: 110, 4: 83},
        "Home/Neutral": {
            0: "San Antonio Spurs",
            1: "Golden State Warriors",
            2: "Los Angeles Lakers",
            3: "Indiana Pacers",
            4: "Orlando Magic",
        },
        "PTS.1": {0: 106, 1: 96, 2: 93, 3: 119, 4: 102},
        "Unnamed: 6": {
            0: "Box Score",
            1: "Box Score",
            2: "Box Score",
            3: "Box Score",
            4: "Box Score",
        },
        "Attend.": {0: 18797, 1: 19832, 2: 18997, 3: 16212, 4: 17519},
    }
)

blah["blawdy"] = np.where(blah.index > 2, 1, 0)

blah

In [ ]:
# assert False

In [ ]:
master["Start (ET)"].value_counts()

In [ ]:
master[master["Start (ET)"] == "Playoffs"]["link"].values

### Scratch


In [ ]:
import pandas as pd

pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [ ]:
# df = pd.read_csv("../data/br_all_games.csv")

# df["game_time"] = pd.to_datetime(df["game_time"])
# print(df.shape)

# df.head()

In [ ]:
# temp = df[
#     (df["game_time"] >= "2020-07-30") & (df["game_time"] <= "2020-07-31")
# ].sort_values(by=["team", "game_time"])
# print(temp.shape)
# temp

In [ ]:
# a = pd.read_html(
#     "https://www.basketball-reference.com/leagues/NBA_2020_games-august.html"
# )
# print(len(a))
# a[0]

In [ ]:
master.info()

In [ ]:
new_dfs = []
for df in dfs:
    temp_df = df.copy()
    temp_df['Notes'] = temp_df['Notes'].astype(str)
    is_playoffs = False
    shape_string = ""
    playoff_ind = 1_000_000

    if temp_df[temp_df["Date"].str.lower() == "playoffs"].shape[0] > 0:
        is_playoffs = True
        playoff_ind = temp_df[temp_df["Date"].str.lower() == "playoffs"].index[0]
        temp_df["is_playoff"] = np.where(temp_df.index >= playoff_ind, 1, 0)
    elif temp_df[temp_df["Notes"].str.lower() == "play-in game"].shape[0] > 0:
        is_playoffs = True
        playoff_ind = temp_df[temp_df["Notes"].str.lower() == "play-in game"].index[0]
        temp_df["is_playoff"] = np.where(temp_df.index >= playoff_ind, 1, 0)
    elif is_playoffs:
        temp_df["is_playoff"] = 1
    else:
        temp_df["is_playoff"] = 0

    temp_df["season_end_year"] = ["Date"].str.split(",").str[-1].astype(int)[-1]
    # temp_df["season_end_year"] = season
    # temp_df["season_url"] = temp_url
    # temp_df["link"] = l
    # shape_string += " " + str(temp_df.shape)

    new_dfs.append(temp_df)
    

print(len(new_dfs))

In [ ]:
master = pd.concat(new_dfs).reset_index(drop=True)
print(master.shape)

In [ ]:
dfs[14]["Date"].str.split(",").str[-1].astype(int)